In [12]:
import pandas as pd
df = pd.read_csv('/Users/north/Documents/GR5243/project/5243project/AB_NYC_2019.csv')
df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365


logit:根据客户喜爱程度，filter/classify data， 找到key element
    根据以上建模，预测price
Data Cleaning:
-replace NA value to 0
-drop id, host_id, host_name

Data scaning
-plot all numberic vraibles relationship: scatter plot matrix(like R)
-

Data Analysis
-Finding relationship between number of review, reviews_permonth, and availability_365 (continue with data scaning, sub plot or regression)

-using name columns: tokenized, anaylsis which character is more poppular to customer ((number of review) * (review per month) as temp popularity column)
--finding most popular 8 string first, and create new column
--ex: new column = 'Sunshie', if exist in value=1

-using latitude and longitude to separate housing to group: uptown, midtown, downtown, etc: check popularity based on location: avg price per night per group
-check price fluctuation based on location
-using regression analysis the relationship between popularity and interested factors
-which neighbourhood/group is most attractive [which is more accurate]
-review frequency and availability,( did the review are all come from extreem review, low availability may means high popularity)
-using new york city.png plot the desenty of airbnb listing

-Logistic reg: y=price, x =  attractive charactor in name, group, neigborhood, neighbourhood_group,minimum_nights